# Regressão linear: Validação cruzada e avaliação final

Finalmente, iremos formar um modelo de regressão linear completa, aplicando o pré-processamento de dados, verificando o modelo por validação cruzada, avaliando-o num subset de testes e, finalmente, fazendo previsões com ele.

Este é, portanto, um exemplo completo de como formar um modelo de regressão linear multivariável.

## O que vamos fazer?
- Criar um dataset sintético para regressão linear multivariável 
- Reprocessar os dados
- Formar o modelo no subset de formação e verificar a sua adequação 
- Encontrar o hiper-parâmetro lambda ótimo no subset de validação cruzada ou CV 
- Avaliar o modelo no subset de teste
- Fazer previsões sobre novos exemplos

In [20]:
import time
import numpy as np
from matplotlib import pyplot as plt

## Criar um dataset sintético para regressão linear

Vamos começar, como habitualmente, por criar um dataset sintético para este exercício

Criar um manualmente com um termo de erro modificável, não esquecendo o seu termo de bias

In [21]:
# TODO: Gerar um dataset sintéticos manualmente, com o termo de bias e o termo de erro

m = 1000
n = 3

# Criar uma matriz de números aleatórios no intervalo [-1, 1)
X = np.random.uniform(-1, 1, (m, n))
# Inserir um vetor de 1s como primeira coluna de X
X = np.insert(X, 0, 1, axis=1)

Theta_verd = np.random.rand(n + 1) * 10 

# Computar Y multiplicando os vetores X e Theta 
Y = np.matmul(X, Theta_verd)

error = 0.2

# Calcular Y com erro
termino_error = np.random.uniform(-1, 1, size=len(Y)) * error
Y = Y + Y * termino_error

# Comprovar os valores e dimensões dos vetores
print("Dimensao Theta:", Theta_verd.shape)
print("Theta:", Theta_verd)

print()

print('Primeiras 10 filas e 5 colunas de X e Y:')
print("X:", X[:10])
print("Y:", Y[:10]) 

print()

print("Dimensao de X:", X.shape)
print("Dimensao de Y:", Y.shape)

Dimensao Theta: (4,)
Theta: [3.18837502 1.91516479 6.65796855 4.84542686]

Primeiras 10 filas e 5 colunas de X e Y:
X: [[ 1.         -0.56695651 -0.28414695 -0.55657645]
 [ 1.         -0.99336079  0.88374542  0.63568676]
 [ 1.         -0.93501466  0.66492104 -0.89912375]
 [ 1.          0.82906932 -0.43341289 -0.05990409]
 [ 1.          0.48452726  0.76571436 -0.00277069]
 [ 1.         -0.16466784  0.27243874  0.76432147]
 [ 1.          0.0968995  -0.06579606 -0.27267523]
 [ 1.          0.05814736  0.25641192  0.59848575]
 [ 1.          0.27410141  0.59919333 -0.80181373]
 [ 1.         -0.10216436  0.17313255  0.76635028]]
Y: [-2.59940773 11.47446668  1.65346645  1.49374094  9.06329873  9.71500363
  1.29237615  7.20226328  4.25416477  6.58998752]

Dimensao de X: (1000, 4)
Dimensao de Y: (1000,)


## Pré-processar os dados

Vamos pré-processar os dados por completo, para os preparar.

Desta vez, iremos seguir os passos seguintes para pré-processar os dados:
- Reordená-los aleatoriamente. 
- Normalizá-los.
- Dividi-los em subsets de formação, validação cruzada e de teste

### Reordenar o dataset aleatoriamente

Desta vez, vamos utilizar um dataset sintético criado com base em dados aleatórios. Por conseguinte, não seria necessário reordená-los, uma vez que, sendo aleatórios, já se encontram desorganizados por defeito.

No entanto, podemos normalmente deparar-nos com dataset reais cujos dados têm uma certa ordem, um padrão, o que pode confundir a nossa formação.

Portanto, sempre antes de começarmos a tratar os dados, a primeira coisa que fazemos é reordená-los aleatoriamente, principalmente antes de os dividirmos nos subsets de formação, CV e teste.

*Nota*: Muito importante, recordar sempre de reordenar sempre *X* e *Y* na mesma ordem, para que a cada exemplo seja atribuído o mesmo resultado antes e depois da reordenação.

In [22]:
from sklearn.utils import shuffle

# TODO: Reordenar aleatoriamente o dataset

print('Primeiras 10 filas e 5 colunas de X e Y:') 
print("X:", X[:10])
print("Y:", Y[:10])

print('Reordenamos X e Y:')
# Se preferir, pode usar a função sklearn.utils.shuffle convenience function.
# Usar um estado inicial aleatório de 42, de modo a manter a reprodutibilidade.
X, Y = shuffle(X, Y, random_state=42)

print('Primeiras 10 filas e 5 colunas de X e Y:') 
print("X:", X[:10])
print("Y:", Y[:10])

print('Dimensões de X e Y:') 
print(X.shape, Y.shape)

Primeiras 10 filas e 5 colunas de X e Y:
X: [[ 1.         -0.56695651 -0.28414695 -0.55657645]
 [ 1.         -0.99336079  0.88374542  0.63568676]
 [ 1.         -0.93501466  0.66492104 -0.89912375]
 [ 1.          0.82906932 -0.43341289 -0.05990409]
 [ 1.          0.48452726  0.76571436 -0.00277069]
 [ 1.         -0.16466784  0.27243874  0.76432147]
 [ 1.          0.0968995  -0.06579606 -0.27267523]
 [ 1.          0.05814736  0.25641192  0.59848575]
 [ 1.          0.27410141  0.59919333 -0.80181373]
 [ 1.         -0.10216436  0.17313255  0.76635028]]
Y: [-2.59940773 11.47446668  1.65346645  1.49374094  9.06329873  9.71500363
  1.29237615  7.20226328  4.25416477  6.58998752]
Reordenamos X e Y:
Primeiras 10 filas e 5 colunas de X e Y:
X: [[ 1.          0.88466959  0.01199735 -0.17875024]
 [ 1.         -0.45627649 -0.99240467  0.36805799]
 [ 1.          0.49738    -0.8029932   0.46849825]
 [ 1.          0.19790748 -0.40942067  0.68975091]
 [ 1.          0.8346368  -0.51616987  0.98354635]
 

Comprovar se *X* e *Y* têm as dimensões corretas e uma ordem diferente da anterior

## Normalizar o dataset

Uma vez reordenados os dados aleatoriamente, vamos proceder à normalização do dataset de 
exemplos *X*.

Para o fazer, copiar as células de código dos exercícios anteriores para o normalizar.

*Nota*: Em exercícios anteriores utilizávamos 2 células de código diferentes, uma para definir a função de normalização e outra para normalizar o dataset. Pode combinar ambas as células numa única célula para guardar este pré-processamento numa célula reutilizável no futuro

In [24]:
# TODO: Normalizar o dataset ocom uma sua função de normalização.

def normalize(x, mu, std):
    """ Normalizar um dataset com exemplos X
    
    Argumentos posicionais:
    x -- array 2D de Numpy com os exemplos, sem termo de bias
    mu -- vetor 1D de Numpy com a média de cada característica/coluna
    std -- vetor 1D de Numpy com o desvio típico de cada característica/coluna
    
    Devolver:
    X norm -- array 2D de Numpy com os exemplos, com as suas características normalizadas 
    """
    return [...]

# Encontrar a média e o desvio padrão das características de X (colunas), exceto a primeira (parcialidade).
mu = []
for i in range(len(X[0])-1):
    mu.append(np.mean(X[i+1]))
print(mu)
std = [...]

print('X original:') 
print(X) 
print(X.shape)

print('Média e desvio típico das características:') 
print(mu)
print(mu.shape) 
print(std) 
print(std.shape)

print('X normalizada:') 
X_norm = np.copy(X)
X_norm[...] = normalize(X[...], mu, std) # Normalizar apenas a coluna 1 e as colunas seguintes, não a coluna 0.
print(X_norm) 
print(X_norm.shape)

[-0.020155794424534745, 0.29072126249360203, 0.36955942946206044]
X original:
[[ 1.          0.88466959  0.01199735 -0.17875024]
 [ 1.         -0.45627649 -0.99240467  0.36805799]
 [ 1.          0.49738    -0.8029932   0.46849825]
 ...
 [ 1.          0.68863513 -0.40342078  0.28564326]
 [ 1.         -0.9095763   0.5203206  -0.40124442]
 [ 1.         -0.0041393   0.88953358  0.65216492]]
(1000, 4)
Média e desvio típico das características:
[-0.020155794424534745, 0.29072126249360203, 0.36955942946206044]


AttributeError: 'list' object has no attribute 'shape'

*Nota*: Algumas pessoas preferem calcular a média *mu* e o desvio padrão *std* de cada característica/coluna de *X* na mesma função de normalização se não forem incluídos como argumentos (o que seria então opcional), devolvendo os 3 valores, uma vez que para fazer previsões precisamos de normalizar os dados com a mesma *mu* e *std* originais.

Se preferir, pode modificar a sua implementação da função para o fazer.

### Dividir os dataset e subset de formação, CV e testes

Finalmente, vamos dividir o dataset nos 3 subset a serem utilizados.

Para isso, vamos utilizar uma proporção de 60%/20%/20%, uma vez que estamos a partir de 1 000 exemplos. Como dissemos, para um número diferente de exemplos, podemos modificar a proporção:

In [ ]:
# TODO: Dividir o dataset X e Y nos 3 subconjuntos, de acordo com os ratios indicados.

ratios = [60,20,20]
print('Ratios:\n', ratios, ratios[0] + ratios[1] + ratios[2])

r = [0,0]
# Dica: a função round() e o atributo x.shape podem ser úteis para si
r[0] = [...]
r[1] = [...]
print('Índices de corte:\n', r)

# Dica: a função np.array_split() pode ser útil para si
X_train, X_cv, X_test = [...] 
Y_train, Y_cv, Y_test = [...]

print('Tamanhos dos subsets:') 
print(X_train.shape) 
print(Y_train.shape) 
print(X_cv.shape) print(Y_cv.shape) 
print(X_test.shape) 
print(Y_test.shape)

## Formar um modelo inicial sobre o subset de formação.

Antes de começarmos a otimizar o hiper-parâmetro *lambda*, iremos formar um modelo inicial não regularizado no subset de formação, para verificar o seu desempenho e adequação, e para ter a certeza de que faz sentido formar um modelo ML de regressão linear multivariável em tal dataset, uma vez que as características podem não ser adequadas, pode haver uma relação baixa entre elas, podem não seguir uma relação linear, etc.


Para o fazer, vamos seguir os passos seguintes:
- Formar um modelo inicial, sem regularização, com *lambda* a 0.
- Representar o histórico do seu custo para comprovar a sua evolução. 
- Voltar a formar o modelo se necessário, por exemplo, variando o ratio de aprendizagem *alpha*.

Copia las celdas de ejercicios anteriores donde implementabas las funciones de coste y gradient descent regularizadas, y copia la celda donde entrenabas el modelo:

In [ ]:
# TODO: Copiar as células com custo regularizado e funções de gradient descent

In [ ]:
# TODO: Copiar a célula onde formamos o modelo
# Formar o seu modelo no subconjunto de formação não regularizada.

Da mesma forma que anteriormente, verificar a formação do modelo, representando graficamente a evolução da função de custo de acordo com o número de iterações, copiando a célula de código correspondente:

In [ ]:
# TODO: Representar a evolução da função de custo vs o número de iterações

plt.figure(1)

Como dissemos anteriormente, reveja a formação do seu modelo e modifique alguns parâmetros, se necessário, para o voltar a formar, procurando um bom desempenho: o ratio de aprendizagem, o ponto de convergência, o número máximo de iterações, etc., com exceção do parâmetro de regularização *lambda*, que deve ser fixado em 0.

*Nota*: Este ponto é importante, visto que estes hiper-parâmetros serão geralmente os mesmos que utilizaremos para o resto da 
otimização do modelo, pelo que agora é o momento de encontrar os valores certos

### Comprovar se existe desvio ou sobreajuste, *bias* ou *variância*

Há um teste que podemos fazer rapidamente para verificar se o nosso modelo inicial sofre claramente de parcialidade, variação, ou se tem um desempenho mais ou menos aceitável.

Vamos representar graficamente a evolução da função de custo de 2 modelos, um formado sobre os primeiros n exemplos do subset de formação e o outro formado sobre os primeiros n exemplos do subset de validação cruzada.

Uma vez que o subset de formação e o subset de validação cruzada não têm o mesmo tamanho, utiliza apenas o mesmo número de exemplos para este subset do que o número total de exemplos no subset CV.

Para o fazer, formar 2 modelos em condições de igualdade, copiando novamente as células de código correspondentes :

In [ ]:
#  TODO: Estabelecer um theta_ini e híper-parâmetros comuns para ambos os modelos, a fim de os formar em igualdade de
# condições

theta_ini = [...]

print('Theta inicial:') 
print(theta_ini)

alpha = 1e-1 
lambda_ = 0. 
e = 1e-3 
iter_ = 1e3

print('Hiper-parâmetros usados:')
print('Alpha:', alpha, 'Error máx.:', e, 'Nº iter', iter_)

In [ ]:
#  TODO: Formar um modelo sem regularização nos primeiros n valores do X_train, onde n é o número de
# exemplos disponíveis em X_cv
# Usar j_hist_train e theta_train como nomes de variáveis para os distinguir do outro modelo.

*Nota*: Comprovar se o theta_ini não foi modificado, ou modificar o seu código para que ambos os modelos utilizem o mesmo theta_ini:

In [ ]:
# TODO: Da mesma forma, formar um modelo sem regularização em X_cv com os mesmos parâmetros.
# Recordar de usar j_hist_cv e theta_cv como nomes de variável.

Agora representar ambas as evoluções no mesmo gráfico, com cores diferentes:

In [ ]:
# TODO: Representar num gráfico de linhas ambas as evoluções para comparação.

plt.figure(2)

plt.title() 
plt.xlabel() 
plt.ylabel()

# Usa colores diferentes para ambas series, e indica una legenda para os distinguir
plt.plot() 
plt.plot()

plt.show()

Com um dataset sintético aleatórios é difícil que um ou outro seja o caso, mas nesta forma poderíamos apreciar tais problemas da seguinte forma:

Se o custo final em ambos os subset for elevado, pode haver um problema de desvio ou *bias*.
Se o custo final em ambos os subset for muito diferente um do outro, pode haver um problema de sobreajuste ou *variação*.

Recordar o que ambos significavam:
- O desvio ocorre quando o modelo não consegue ajustar suficientemente bem na curva do dataset, ou porque não são as características certas (ou porque faltam outras), ou porque os dados têm demasiados erros, ou porque o modelo segue uma relação diferente ou é demasiado simples.
- O sobreajuste ocorre quando o modelo se ajusta demasiado bem à curva do dataset, demasiado bem, demasiado próximo dos exemplos sobre os quais foi formado, e quando tem de prever novos resultados, não o faz corretamente.

### Comprovar a adequação do modelo

Como dissemos, outro motivo para formar um modelo inicial é comprovar se faz sentido formar um modelo de regressão linear multivariável nesse dataset. Se virmos que o modelo sofre de sobreajuste, podemos sempre o corrigir com a regularização. No entanto, se virmos que sofre de um elevado 

Se virmos que o modelo sofre de sobreajuste, podemos sempre o corrigir com a regularização. No entanto, se virmos que sofre de um elevado desvio, ou seja, que o custo final é muito elevado, o nosso tipo de modelo ou as características escolhidas podem não ser adequados para este problema.

Neste caso, descobrimos que o erro é suficientemente baixo para o tornar promissor para continuar a formar um modelo de regressão linear multivariável como este

## Encontrar o hiper-parâmetro lambda ótimo no subset de validação cruzada

Agora, a fim de encontrar o lambda ideal, iremos formar um modelo diferente para cada valor de lambda a ser considerado, no subset de formação, e verificar a sua exatidão no subset de validação cruzada. 

Vamos representar graficamente o erro ou custo final de cada modelo vs o valor de lambda usado, para ver qual modelo tem um erro ou custo inferior no subset de validação cruzada.

Deste modo, formamos todos os modelos no mesmo subset e em condições iguais (exceto lambda), e avaliamo-los num subset de dados que não tenham visto anteriormente, que não utilizámos para os formar.

O subset CV não é, portanto, utilizado para formar o modelo, mas apenas para avaliar o valor lambda ótimo. Exceto, como fizemos no ponto anterior, para fazer uma rápida avaliação inicial da possível ocorrência de sobreajuste.

In [ ]:
# TODO: Formar um modelo para cada valor lambda diferente em X_train e avaliá-lo em X_cv

lambdas = [0., 1e-3, 3e-3, 1e-2, 3e-2, 1e-1, 3e-1, 1e0, 3e0, 1e1]

# Completar o código para formar um modelo diferente para cada valor de lambda no X_train
# Armazenar o theta e erro/custo final
# Posteriormente, avaliar o seu custo total no subset da CV

# Armazenar essa informação nas seguintes matrizes, do mesmo tamanho que os lambdas
j_train = [...]
j_cv = [...]
theta_cv = [...]

Uma vez todos os modelos formados, representar num gráfico de linhas o seu custo final sobre o subset de formação e o custo final sobre o subset de CV vs o valor *lambda* utilizado:

In [ ]:
# TODO: Representar graficamente o erro final para cada valor de lambda

plt.figure(3)

# Completar com o seu código

Uma vez representados estes erros finais, podemos escolher manualmente o modelo com o valor *lambda* ótimo, ou podemos fazê-lo de forma automatizada com código

In [ ]:
# TODO: Escolher o modelo ótimo e o valor lambda, com o menor erro no subset do CV

# Iterar sobre todas as combinações de theta e lambda e escolher o custo mais baixo no subset do CV

j_final = [...] 
theta_final = [...] 
lambda_final = [...]

Uma vez implementadas todas as etapas acima mencionadas, temos o nosso modelo formado e os seus hiper -parâmetros otimizados.

## Avaliar o modelo finalmente sobre o subset de teste

Finalmente, encontrámos os nossos coeficientes theta e hiper-parâmetro lambda ótimos, pelo que temos agora um modelo formado pronto a ser usado.

No entanto, embora tenhamos calculado o seu erro ou custo final no subset CV, utilizámos este subset para escolher o modelo, para terminar a “Formação”, para atuar sobre o modelo. Portanto, ainda não testamos como este modelo irá funcionar com dados que nunca viu antes.

Por isso, vamos finalmente avaliá-lo no subset de teste, num subset que ainda não utilizámos, nem para formar o modelo, nem para escolher os seus hiper-parâmetros. Um subset separado que a formação do modelo ainda não viu.

Para tal, vamos calcular o erro ou custo total no subset de teste e verificar graficamente os resíduos do modelo no mesmo

In [ ]:
# TODO: Calcular o erro do modelo no subset de teste usando a função de custo com a correspondente
# theta e lambda

j_test = [...]

In [ ]:
# TODO: Calcular as previsões do modelo no subset de teste, calcular os resíduos e representá-los

Y_test_pred = [...]

residuos = [...]

plt.figure(4)

# Completar com o seu código

plt.show()

Desta forma, podemos ter uma ideia mais realista da precisão do nosso modelo e do seu comportamento com novos exemplos no futuro.

## Fazer previsões sobre novos exemplos

Com o nosso modelo formado, otimizado e avaliado, tudo o que resta é pô-lo a funcionar, fazendo previsões com novos exemplos.

Para isso, vamos:
- Gerar um novo exemplo seguindo o mesmo padrão que o dataset original. 
- Normalizar as suas características antes de poder fazer previsões sobre eles. 
- Gerar uma previsão para esse novo exemplo

In [ ]:
#  TODO: Gerar um novo exemplo seguindo o padrão original, com termo de bias e erro aleatório.

X_pred = [...]

# Normalizar as suas características (exceto o termo de bias) com as médias e desvios típicos originais
X_pred = [...]

# Gerar uma previsão para esse exemplo.
Y_pred = [...]